# Preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [ ]:
def pull_model(local_llm):
    !ollama pull local_llm

In [ ]:
def start_model(local_llm):
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [2]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# response: biased or non-biased
bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


# Bias detection

## Dati

https://www.kaggle.com/datasets/timospinde/mbic-a-media-bias-annotation-dataset (vedi anche paper di riferimento MBIC)

In [4]:
df = pd.read_excel("/content/drive/MyDrive/predictions/labeled_dataset.xlsx")

print(len(df))

print(df["Label_bias"].unique()) #ci sono tre valori, i no-agreement come li gestiamo?
# eliminiamo i no-agreement (non c'è accordo che siano biased o non-biased).

df = df[df["Label_bias"] != "No agreement"]

print(len(df))

1700
['Biased' 'Non-biased' 'No agreement']
1551


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
#all'incirca 1551 testi di test, news biased e non biased

# Biased = 0
# Non-biased = 1

x_test = df["sentence"]
y_test_pre = df["Label_bias"] #Biased #Non-biased

print(len(x_test))
print(len(y_test_pre))

label_biased = df[df["Label_bias"]=="Biased"] #Label 0
label_non_biased = df[df["Label_bias"]=="Non-biased"] #Label 1
print(f"Label con valore Biased: {len(label_biased)}")
print(f"Label con valore Non-biased: {len(label_non_biased)}")

1551
1551
Label con valore Biased: 1018
Label con valore Non-biased: 533


In [6]:
y_test = [0 if label=="Biased" else 1 for label in y_test_pre]

print(f"Label con valore Biased: {y_test.count(0)}")
print(f"Label con valore Non-biased: {y_test.count(1)}")

Label con valore Biased: 1018
Label con valore Non-biased: 533


## Test

In [ ]:
examples = [
    {"input": "I'm not racist, but [group] are just not as smart as us.", "output": "biased"},
    {"input": "Women should know their place.", "output": "biased"},
    {"input": "She excelled in her exams due to her dedication and hard work.", "output": "no-biased"},
    {"input": "People from that country are lazy and unproductive.", "output": "biased"},
    {"input": "He was selected for the role based on his qualifications and experience.", "output": "non-biased"},
    {"input": "Students from that school always perform poorly.", "output": "biased"},
    {"input": "The decision was made after a thorough review of all the candidates.", "output": "no-biased"},
    {"input": "He's not suitable for the job because he's too old.", "output": "biased"},
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

def create_prompt(shot):
    example_prompt  = ChatPromptTemplate.from_messages(
        [
            ("user", "{input}"),
            ("assistant", "{output}"),
        ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt ,
        examples=examples[:shot], #:2 two-shot, #4: four-shot #8 eight-shot
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You will be provided with a text, and your task is to classify it into
                        one of two categories: "biased" or "non-biased." A text is considered "biased"
                        if it expresses a personal opinion, uses emotional or partial language, or promotes
                        a particular perspective at the expense of others. A text is considered "non-biased"
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
             few_shot_prompt,
            ("user", "{input}"),
        ] if shot>0 else
        [
            ("system", """You will be provided with a text, and your task is to classify it into
                        one of two categories: "biased" or "non-biased." A text is considered "biased"
                        if it expresses a personal opinion, uses emotional or partial language, or promotes
                        a particular perspective at the expense of others. A text is considered "non-biased"
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
            ("user", "{input}"),
        ]
    )
    return prompt

In [ ]:
# response: biased or non-biased

def bias_det(llm, shot):
    prompt_final = create_prompt(shot)
    bias_det_chain = prompt_final | llm
    return bias_det_chain

In [ ]:
# Conferma label
# Label 0: biased
# Label 1: non-biased

def predict(llm,x_test,shot,encoder):
    y_pred = []
    y_pred_label = []
    if encoder:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
    else:
        chain = bias_det(llm,shot)
    for x in tqdm(x_test):
        if encoder:
            answer = bias_detection(x)[0]["label"]
        else:
            answer = chain.invoke({"input": x})
        if "non-biased" in answer.lower(): y_pred.append(1)
        else: y_pred.append(0)
        y_pred_label.append(answer) #per verificare che le risposte siano sensate
    return y_pred, y_pred_label

# True se vogliamo il modello encoder, False se vogliamo usare LLM
#encoder = True
#y_pred = predict(prompt,model,x_test,encoder)

In [ ]:
import time
import json

models = ["llama3.1","gemma2","mistral"]
shots = [0,2,4,8]


def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

for model in models:
    if model == "encoder":
        y_pred, y_pred_label = predict(None,x_test,0,True)
        write_file(f"/content/drive/MyDrive/predictions/prediction_bias_encoder.json", y_pred)
    else:
        start_ollama()
        pull_model(model)
        start_model(model)
        time.sleep(500)
        llm = Ollama(model=model, temperature=0)
        for shot in shots:
            y_pred, y_pred_label = predict(llm,x_test,shot,False)
            if model=="llama3.1":
                write_file(f"/content/drive/MyDrive/predictions/prediction_bias_llama31_{shot}_shot.json", y_pred)
                write_file(f"/content/drive/MyDrive/predictions/prediction_label_bias_llama31_{shot}_shot.json", y_pred_label)
            else:
                write_file(f"/content/drive/MyDrive/predictions/prediction_bias_{model}_{shot}_shot.json", y_pred)
                write_file(f"/content/drive/MyDrive/predictions/prediction_label_bias_{model}_{shot}_shot.json", y_pred_label)

In [15]:
# Valutazione del modello
#Label 0: biased
#Label 1: non biased
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

y_pred = load_json("/content/drive/MyDrive/predictions/prediction_bias_0_shot.json")

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

Accuracy: 0.778207607994842
Confusion Matrix:
[[826 192]
 [152 381]]
Classification Report:
{'0': {'precision': 0.8445807770961146, 'recall': 0.8113948919449901, 'f1-score': 0.8276553106212425, 'support': 1018.0}, '1': {'precision': 0.6649214659685864, 'recall': 0.7148217636022514, 'f1-score': 0.6889692585895116, 'support': 533.0}, 'accuracy': 0.778207607994842, 'macro avg': {'precision': 0.7547511215323505, 'recall': 0.7631083277736208, 'f1-score': 0.7583122846053771, 'support': 1551.0}, 'weighted avg': {'precision': 0.7828409880368157, 'recall': 0.778207607994842, 'f1-score': 0.7799959516702996, 'support': 1551.0}}


In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)


#dict_to_write = {"class_report":class_report, "0_0":int(conf_matrix[0][0]), "0_1":int(conf_matrix[0][1]), "1_0":int(conf_matrix[1][0]), "1_1":int(conf_matrix[1][1])}
#write_file("/kaggle/working/test_bias_mistral_zero_shot.json", dict_to_write)

#y_pred = [0,1,0]
#write_file("/content/drive/MyDrive/predictions/prova.json", y_pred)

# Debiasing

In [ ]:
#todo predisposizione dati e metriche di utilizzo

In [ ]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes.
         <|eot_id|><|start_header_id|>user<|end_header_id|>
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    debiasing = prompt | llm | StrOutputParser()
    return debiasing